In [110]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor
from geopy.distance import great_circle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import requests

In [111]:
# 1. 데이터 로드
facilities_capacity = pd.read_csv("C:/kepco/sprint2/데이터/최종/설비용량.csv")
power_usage = pd.read_csv("C:/kepco/sprint2/데이터/최종/전력사용량.csv")
weather = pd.read_csv("C:/kepco/sprint2/데이터/최종/진짜기상.csv")

In [112]:
facilities_capacity

,capacity,address,latitude,longitude
0,197.64,강원특별자치도 고성군 현내면 산학리 산 57번지 6호,38.487784,128.397384
1,99.00,강원특별자치도 고성군 현내면 산학리 산 57번지 6호,38.487784,128.397384
2,393.60,강원특별자치도 고성군 토성면 용촌리 555번지,38.236106,128.570488
3,83.20,강원특별자치도 고성군 거진읍 자산길 19,38.438013,128.445395
4,97.20,강원특별자치도 고성군 거진읍 봉평리 596-1,38.441105,128.433905
...,...,...,...,...
36121,99.00,"충청북도 보은군 장안면 봉비길 84-32, 축사",36.460978,127.797865
36122,99.96,충청북도 보은군 마로면 송현2길 4-14,36.433270,127.834969
36123,99.96,"충청북도 보은군 마로면 갈평길 217, 축사",36.446429,127.839726
36124,99.96,충청북도 보은군 마로면 수문불목로 83,36.443400,127.814388


In [113]:
power_usage

,year,month,metro,city,houseCnt,powerUsage,bill
0,2022,1,서울특별시,성동구,112001,247.53,27933
1,2022,1,서울특별시,성북구,173553,233.35,27977
2,2022,1,서울특별시,동작구,172987,213.16,24532
3,2022,1,서울특별시,영등포구,173461,225.24,25401
4,2022,1,서울특별시,관악구,286468,190.35,21188
...,...,...,...,...,...,...,...
5565,2023,10,제주특별자치도,서귀포시,107099,205.49,32612
5566,2023,11,제주특별자치도,제주시,274284,189.29,27735
5567,2023,11,제주특별자치도,서귀포시,107509,179.07,27285
5568,2023,12,제주특별자치도,서귀포시,107631,199.02,30682


In [114]:
weather

,지점,지점명,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),합계 일사량(MJ/m2),평균 전운량(1/10),일출,일몰,일조시간,latitude,longitude
0,105,강릉,2023-01-01,4.8,1.0,10.2,0.0,2.9,31.6,9.85,1.6,07:38,17:58,10.333333,37.75147,128.89099
1,105,강릉,2023-01-02,1.6,-2.0,7.7,0.0,2.6,25.5,10.45,1.4,07:37,17:59,10.366667,37.75147,128.89099
2,105,강릉,2023-01-03,1.3,-1.6,7.5,0.0,3.6,25.5,10.81,0.5,07:49,17:28,9.650000,37.75147,128.89099
3,105,강릉,2023-01-04,2.5,-1.3,8.0,0.0,3.2,21.9,10.54,0.4,07:49,17:29,9.666667,37.75147,128.89099
4,105,강릉,2023-01-05,3.6,-2.1,10.3,0.0,2.4,32.4,10.16,1.0,07:49,17:30,9.683333,37.75147,128.89099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58929,169,흑산도,2024-08-27,27.5,25.3,31.3,0.0,3.2,86.9,13.06,9.4,07:35,20:08,12.550000,34.68719,125.45105
58930,169,흑산도,2024-08-28,28.0,23.9,31.0,0.0,6.1,79.0,24.22,4.3,07:34,20:06,12.533333,34.68719,125.45105
58931,169,흑산도,2024-08-29,27.8,25.2,31.4,0.0,7.3,76.9,25.69,0.0,07:32,20:04,12.533333,34.68719,125.45105
58932,169,흑산도,2024-08-30,26.8,24.0,30.3,0.0,7.9,82.3,20.63,2.1,07:31,20:03,12.533333,34.68719,125.45105


In [115]:
# 사용자 주소 받아서 위도 경도로 변환하기
address = input("주소를 입력하세요: ")

def get_lat_lon(address):
    API_KEY = 'api key'
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': address, 'key': API_KEY}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        results = response.json()
        if results['status'] == 'OK':
            location = results['results'][0]['geometry']['location']
            return location['lat'], location['lng']
    return None, None

user_lat, user_lng = get_lat_lon(address)
get_lat_lon(address)

주소를 입력하세요:  전주 한옥마을


(35.8175376, 127.1520417)

In [116]:
# 좌표로 거리 찾기 함수 생성
from math import radians, cos, sin, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    # 두 지점 사이의 Haversine 거리를 계산하는 함수
    R = 6371.0  # 지구의 반지름 (킬로미터 단위)
    dlat = radians(lat2 - lat1)  # 위도의 차이를 라디안으로 변환
    dlon = radians(lon2 - lon1)  # 경도의 차이를 라디안으로 변환
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2  
    # Haversine 공식을 사용하여 a를 계산
    c = 2 * atan2(sqrt(a), sqrt(1 - a))  # Haversine 공식의 c를 계산
    distance = R * c  # 거리 계산
    return distance  # 계산된 거리를 반환합니다.

In [141]:
# 위도와 경도를 숫자로 변환
facilities_capacity['latitude'] = pd.to_numeric(facilities_capacity['latitude'], errors='coerce')
facilities_capacity['longitude'] = pd.to_numeric(facilities_capacity['longitude'], errors='coerce')

# 중복된 위도와 경도를 가진 행 제거 (첫 번째로 나타난 행만 유지)
unique_facilities = facilities_capacity.drop_duplicates(subset=['latitude', 'longitude']).copy()

# 사용자 위치(user_lat, user_lng)가 정의되어 있는 경우
if user_lat and user_lng:
    # 각 공공시설과 사용자 간의 거리를 계산 (apply를 사용하여 행별로 계산)
    unique_facilities.loc[:, 'distance'] = unique_facilities.apply(
        lambda row: haversine(user_lat, user_lng, row['latitude'], row['longitude']),
        axis=1
    )
    
    # 거리 기준으로 상위 5개의 가장 가까운 공공시설을 선택
    top_5_facilities = unique_facilities.nsmallest(5, 'distance')

    
print("가장 가까운 상위 5개 공공시설:")
top_5_facilities

가장 가까운 상위 5개 공공시설:


,capacity,address,latitude,longitude,distance
15424,97.94,"전라북도 전주시 완산구 강변로 242, 101동 1709호 (효자동1가, 남양아이좋...",35.811276,127.112866,3.600458
16516,99.00,"전라북도 익산시 선화로10길 17(모현동1가, 익산e편한세상)",35.946449,126.944530,23.558232
21677,177.60,"대야면 접산리 1070-5 건물상부(주5, 주6, 주7, 주8)",35.933848,126.876536,27.990223
21510,50.32,군산시 대야면 접산리 24-1,35.952192,126.887770,28.125010
24080,155.20,충청남도 논산시 연무읍 고내리 265,36.070577,127.129141,28.212169


In [118]:
# 결측값 확인
weather.isnull().sum()

지점                 0
지점명                0
일시                 0
평균기온(°C)           0
최저기온(°C)           0
최고기온(°C)           0
일강수량(mm)           0
평균 풍속(m/s)       138
평균 상대습도(%)       140
합계 일사량(MJ/m2)      0
평균 전운량(1/10)      42
일출                 0
일몰                 0
일조시간               0
latitude           0
longitude          0
dtype: int64

In [119]:
# 결측값 전처리
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# null 값이 있는 열들 확인
cols_with_missing = ['평균 풍속(m/s)', '평균 상대습도(%)', '평균 전운량(1/10)']

# null 값이 있는 열들을 예측하기 위한 feature 선정
features = ['평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '일강수량(mm)', '합계 일사량(MJ/m2)', '일조시간']

# 결측값이 있는 각 열에 대해 반복적으로 예측
for col in cols_with_missing:
    # 결측값이 아닌 행들만 사용하여 학습 데이터 준비
    data = weather[weather[col].notnull()]
    
    X = data[features]
    y = data[col]
    
    # 학습 및 테스트 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 랜덤 포레스트 회귀 모델로 학습
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    
    # 결측값 예측
    null_data = weather[weather[col].isnull()]
    predicted_values = model.predict(null_data[features])
    
    # 예측된 값으로 결측값 대체
    weather.loc[weather[col].isnull(), col] = predicted_values

# 결측값이 잘 채워졌는지 확인
print(weather.isnull().sum())


지점               0
지점명              0
일시               0
평균기온(°C)         0
최저기온(°C)         0
최고기온(°C)         0
일강수량(mm)         0
평균 풍속(m/s)       0
평균 상대습도(%)       0
합계 일사량(MJ/m2)    0
평균 전운량(1/10)     0
일출               0
일몰               0
일조시간             0
latitude         0
longitude        0
dtype: int64


In [121]:
# 발전량 계산 (임의의 비례 상수를 곱해 발전량을 추정)
# 실제로는 패널의 효율, 면적 등을 고려해야 하지만, 여기서는 간단한 예시로 0.2 상수를 곱함
weather['발전량(kWh)'] = weather['합계 일사량(MJ/m2)'] * 0.2  # 발전량 단위는 kWh로 가정

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error, root_mean_squared_error

# 피처와 타겟 정의 (발전량은 예측 대상, 나머지 컬럼은 피처)
X = weather[['평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '일강수량(mm)', '평균 풍속(m/s)', '평균 상대습도(%)', '평균 전운량(1/10)', '일조시간']]
y = weather['발전량(kWh)']

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# DMatrix 형식으로 변환 (XGBoost에서 사용하는 데이터 구조)
train_data = xgb.DMatrix(X_train, label=y_train)
test_data = xgb.DMatrix(X_test, label=y_test)

# XGBoost 모델 설정
params = {
    'objective': 'reg:squarederror',  # 회귀 문제이므로 제곱 오차를 최적화 목표로 설정
    'eval_metric': 'rmse',  # 평가 기준은 RMSE
    'learning_rate': 0.1,
    'max_depth': 6,
    'alpha': 10
}

# 모델 학습
xgb_model = xgb.train(params, train_data, num_boost_round=100)

# 테스트 데이터로 예측
y_pred = xgb_model.predict(test_data)

# # 예측 결과 평가 (평균 제곱근 오차)
# rmse = mean_squared_error(y_test, y_pred, squared=False)
# print(f"RMSE: {rmse}")

# 기존 코드 수정: root_mean_squared_error 사용
rmse = root_mean_squared_error(y_test, y_pred)
print(f"RMSE: {rmse}")

# 테스트 데이터에 발전량 예측값 추가
df_test = X_test.copy()
df_test['실제 발전량(kWh)'] = y_test
df_test['예측 발전량(kWh)'] = y_pred


df_test

RMSE: 3.24014813346236


,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),일조시간,실제 발전량(kWh),예측 발전량(kWh)
53414,26.8,24.5,30.9,0.0,2.6,67.8,6.4,12.616667,3.770000,4.015595
49901,14.3,3.1,23.8,1.4,2.6,60.6,6.0,12.983333,3.896000,3.828493
17419,2.5,-3.0,10.3,0.0,2.3,77.3,0.0,12.550000,2.524000,2.283953
56965,15.8,7.6,24.1,0.0,1.3,76.1,4.1,12.533333,3.939969,4.217438
2726,10.6,2.5,22.5,0.0,0.8,73.8,0.0,11.200000,3.674516,3.508831
...,...,...,...,...,...,...,...,...,...,...
419,0.1,-2.8,2.3,0.7,1.1,85.0,10.0,12.516667,0.184000,0.558197
13661,18.4,13.4,24.9,0.0,1.5,79.0,2.1,12.183333,3.865856,3.734539
5794,7.4,1.3,12.6,0.0,1.8,55.1,5.6,10.300000,2.105639,2.050387
46989,24.6,22.9,28.2,92.9,1.4,95.3,9.3,13.350000,1.277444,1.160202


In [143]:
import folium

lat = top_5_facilities['latitude'].mean()
long = top_5_facilities['longitude'].mean()


m=folium.Map([lat,long],zoom_start=11,tiles='OpenStreetMap')


for i in top_5_facilities.index:
  sub_lat=top_5_facilities.loc[i,'latitude']
  sub_long=top_5_facilities.loc[i,'longitude']

  title=top_5_facilities.loc[i,'address']

  folium.CircleMarker([sub_lat,sub_long],color='red',
                      radius=8,tooltip=title).add_to(m)

# 사용자 핑 추가
folium.CircleMarker([user_lat, user_lng], color='blue',
                    radius=8, tooltip='사용자').add_to(m)

m.save('top_5.html')
m